# BU.330.775 Machine Learning: Design and Deployment
## Lab 4. Model evaluation on MNIST dataset
Jinge Zhou

**Learning Goal:** Evaluate multiple supervised machine learning approaches on the MNIST database

**Background:** The MNIST dataset consists of handwritten digits with 784 features. This dataset was contributed by Yann LeCun, Corinna Cortes, and Christopher J.C. Burges.

## Setup: Import Libraries and Set Default Plot Parameters

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Set up default font sizes for plots
plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

## Step 1: Load and Explore the MNIST Dataset

In [ ]:
from sklearn.datasets import fetch_openml

# Load MNIST dataset
mnist = fetch_openml('mnist_784', as_frame=False)
X, y = mnist.data, mnist.target

print("Shape of X (features):", X.shape)
print("Shape of y (labels):", y.shape)
print("\nFirst 10 labels:", y[:10])

## Step 2: Visualize Sample Digits

In [ ]:
def plot_digit(image_data):
    """Display a single digit image."""
    image = image_data.reshape(28, 28)
    plt.imshow(image, cmap="binary")
    plt.axis("off")

# Display first 100 images in a 10x10 grid
plt.figure(figsize=(9, 9))
for idx, image_data in enumerate(X[:100]):
    plt.subplot(10, 10, idx + 1)
    plot_digit(image_data)
plt.subplots_adjust(wspace=0, hspace=0)
plt.show()

## Step 3: Split Data into Training and Testing Sets

In [ ]:
# Use first 60,000 images for training, remaining 10,000 for testing
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

print("Training set size:", X_train.shape[0])
print("Testing set size:", X_test.shape[0])

## Step 4: Train Binary Classifier (Digit 2 vs Not 2)

In [ ]:
# Create binary labels: True if digit is 2, False otherwise
y_train_2 = (y_train == '2')
y_test_2 = (y_test == '2')

# Train SGD Classifier
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier(random_state=46)
sgd_clf.fit(X_train, y_train_2)

print("Binary classifier trained successfully!")

## Step 5: Evaluate Binary Classifier with Cross-Validation

In [ ]:
from sklearn.model_selection import cross_val_score

# Perform 3-fold cross-validation
cv_scores = cross_val_score(sgd_clf, X_train, y_train_2, cv=3, scoring="accuracy")
print("Cross-validation accuracy scores:", cv_scores)
print("Mean accuracy:", cv_scores.mean())

## Step 6: Generate Confusion Matrix on Test Set

In [ ]:
from sklearn.metrics import confusion_matrix

# Make predictions on test set
y_test_2_pred = sgd_clf.predict(X_test)

# Generate confusion matrix
cm = confusion_matrix(y_test_2, y_test_2_pred)
print("Confusion Matrix:")
print(cm)
print("\nFormat: [[TN, FP],")
print("         [FN, TP]]")

## Homework Question 1 (3pt): Calculate Precision, Recall, and F1-Score

### Part 1: Calculate metrics using sklearn

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Calculate metrics using sklearn
precision = precision_score(y_test_2, y_test_2_pred)
recall = recall_score(y_test_2, y_test_2_pred)
f1 = f1_score(y_test_2, y_test_2_pred)

print("Metrics calculated using sklearn.metrics:")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

### Part 2: Validate using formulas from lecture notes

In [ ]:
# Extract values from confusion matrix
TN, FP, FN, TP = cm[0,0], cm[0,1], cm[1,0], cm[1,1]

print("Confusion Matrix Components:")
print(f"True Negatives (TN): {TN}")
print(f"False Positives (FP): {FP}")
print(f"False Negatives (FN): {FN}")
print(f"True Positives (TP): {TP}")
print()

# Calculate metrics manually using formulas
precision_manual = TP / (TP + FP)
recall_manual = TP / (TP + FN)
f1_manual = 2 * (precision_manual * recall_manual) / (precision_manual + recall_manual)

print("Metrics calculated using formulas:")
print(f"Precision = TP / (TP + FP) = {TP} / ({TP} + {FP}) = {precision_manual:.4f}")
print(f"Recall = TP / (TP + FN) = {TP} / ({TP} + {FN}) = {recall_manual:.4f}")
print(f"F1-Score = 2 × (Precision × Recall) / (Precision + Recall) = {f1_manual:.4f}")
print()

# Verify that manual calculations match sklearn results
print("Verification:")
print(f"Precision match: {np.isclose(precision, precision_manual)}")
print(f"Recall match: {np.isclose(recall, recall_manual)}")
print(f"F1-Score match: {np.isclose(f1, f1_manual)}")

Q1:

The three evaluation metrics for the binary classifier (digit 2 vs not 2) are:

**1. Precision:** This metric measures the accuracy of positive predictions. It tells us what proportion of images predicted as "2" are actually "2". The formula is Precision = TP / (TP + FP), where TP represents true positives and FP represents false positives. A high precision means the classifier rarely misclassifies non-2 digits as 2.

**2. Recall (Sensitivity):** This metric measures the classifier's ability to find all positive instances. It tells us what proportion of actual "2" digits were correctly identified. The formula is Recall = TP / (TP + FN), where FN represents false negatives. A high recall means the classifier rarely misses actual 2 digits.

**3. F1-Score:** This is the harmonic mean of precision and recall, providing a single score that balances both metrics. The formula is F1 = 2 × (Precision × Recall) / (Precision + Recall). This metric is particularly useful when you need to balance the trade-off between precision and recall.

As we can see, the manual calculations perfectly match the sklearn.metrics results, validating our implementation and understanding of these evaluation metrics.

## Step 7: Precision-Recall Trade-off Curve

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import precision_recall_curve

# Get decision scores using cross-validation
y_scores = cross_val_predict(sgd_clf, X_train, y_train_2, cv=3,
                             method="decision_function")

# Calculate precision and recall for different thresholds
precisions, recalls, thresholds = precision_recall_curve(y_train_2, y_scores)

# Plot the trade-off
plt.figure(figsize=(8, 4))
plt.plot(thresholds, precisions[:-1], "b--", label="Precision", linewidth=2)
plt.plot(thresholds, recalls[:-1], "g-", label="Recall", linewidth=2)
plt.axis([-20000, 20000, 0, 1])
plt.grid()
plt.xlabel("Threshold")
plt.legend(loc="center right")
plt.title("Precision-Recall Trade-off")
plt.show()

## Step 8: ROC Curve Analysis

In [ ]:
from sklearn.metrics import roc_curve

# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(y_train_2, y_scores)

# Plot ROC curve
plt.figure(figsize=(6, 5))
plt.plot(fpr, tpr, linewidth=2, label="ROC curve")
plt.plot([0, 1], [0, 1], 'k:', label="Random classifier's ROC curve")
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.grid()
plt.axis([0, 1, 0, 1])
plt.legend(loc="lower right", fontsize=13)
plt.title("ROC Curve")
plt.show()

## Step 9: Calculate AUC Score

In [ ]:
from sklearn.metrics import roc_auc_score

# Calculate Area Under the ROC Curve
auc_score = roc_auc_score(y_train_2, y_scores)
print(f"AUC Score: {auc_score:.4f}")
print(f"\nThe AUC score of {auc_score:.4f} indicates strong classifier performance.")
print("Values closer to 1.0 represent better classification ability.")

## Step 10: Multiclass Classification - Feature Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Scale features to [0, 1] range
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train.astype("float64"))
X_test_scaled = scaler.transform(X_test.astype("float64"))

print("Feature scaling completed.")
print(f"Original feature range: [{X_train.min():.2f}, {X_train.max():.2f}]")
print(f"Scaled feature range: [{X_train_scaled.min():.2f}, {X_train_scaled.max():.2f}]")

## Step 11: Multiclass Classification with SGD

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

# Train multiclass SGD classifier
sgd_clf.fit(X_train_scaled, y_train)

# Make predictions on test set
y_test_pred = sgd_clf.predict(X_test_scaled)

# Display confusion matrix
plt.rc('font', size=9)
ConfusionMatrixDisplay.from_predictions(y_test, y_test_pred)
plt.title("Confusion Matrix - SGD Classifier (Multiclass)")
plt.show()

## Step 12: Normalized Confusion Matrix

In [ ]:
# Display normalized confusion matrix (percentages)
plt.rc('font', size=10)
ConfusionMatrixDisplay.from_predictions(y_test, y_test_pred,
                                        normalize="true", values_format=".0%")
plt.title("Normalized Confusion Matrix - SGD Classifier")
plt.show()

## Step 13: Error Analysis - Weighted Confusion Matrix

In [ ]:
# Create weights for misclassified instances only
sample_weight = (y_test_pred != y_test)

# Display weighted confusion matrix focusing on errors
plt.rc('font', size=10)
ConfusionMatrixDisplay.from_predictions(y_test, y_test_pred,
                                        sample_weight=sample_weight,
                                        normalize="true", values_format=".0%")
plt.title("Weighted Confusion Matrix - Errors Only")
plt.show()

## Extension: Ridge Classifier for Multiclass Classification

In [ ]:
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import accuracy_score, classification_report

# Train Ridge Classifier with optimized hyperparameters
ridge_clf = RidgeClassifier(alpha=0.5, random_state=46)
ridge_clf.fit(X_train_scaled, y_train)

# Make predictions
y_test_pred_ridge = ridge_clf.predict(X_test_scaled)

# Calculate accuracy
ridge_accuracy = accuracy_score(y_test, y_test_pred_ridge)
sgd_accuracy = accuracy_score(y_test, y_test_pred)

print("Model Performance Comparison:")
print(f"Ridge Classifier Accuracy: {ridge_accuracy:.4f}")
print(f"SGD Classifier Accuracy: {sgd_accuracy:.4f}")
print(f"\nImprovement: {(ridge_accuracy - sgd_accuracy)*100:.2f}%")

## Ridge Classifier Confusion Matrix

In [ ]:
# Display normalized confusion matrix for Ridge Classifier
plt.rc('font', size=10)
ConfusionMatrixDisplay.from_predictions(y_test, y_test_pred_ridge,
                                        normalize="true", values_format=".0%")
plt.title("Normalized Confusion Matrix - Ridge Classifier")
plt.show()